In [31]:
import os
import torch

from pathlib import Path 
from transformers import AutoTokenizer
from transformer.config import Config
from main import DemoTransformer

cfg = Config()    
model = DemoTransformer(cfg=cfg)
model_dir = Path('trained_models')
checkpoint_path = max(model_dir.glob('*.pth'), key = os.path.getctime)
model.load_state_dict(torch.load(checkpoint_path))

model.eval()
tokenizer = AutoTokenizer.from_pretrained('gpt2')


/var/folders/77/19fl9pz51mg5qnjlb7f03rbw0000gn/T/ipykernel_65062/3898082890.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_

In [32]:
def run_inference(text: str):
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        output = model(tokens['input_ids'])

    predicted_token_ids = torch.argmax(output, dim=-1)
    predicted_text = tokenizer.decode(predicted_token_ids[0][-1])
    print(f"Input: {text}")
    print(f"Model output: {predicted_text}")

In [40]:
text = "the cat in the"
run_inference(text)

Input: the cat in the
Model output:  hat
